In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 12013.76it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 95.84it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [6]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]


In [7]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [8]:
data = kernelclass.kernel_mtxel()

In [9]:
exc = data["exc"]
print(exc.shape)
exc = exc[:, :, 0, 0, 0, 0]

for row in range(exc.shape[0]):
    row_data = []
    for col in range(exc.shape[1]):
        row_data.append(f"{exc[row, col]:.5f}")
    print(" | ".join(row_data))

(8, 8, 1, 1, 1, 1)
0.47881-0.00000j | -0.02994+0.11418j | -0.07084+0.02386j | -0.06933-0.12470j | -0.10282-0.15727j | 0.01616+0.05548j | 0.02306-0.09088j | 0.00050+0.00007j
-0.02994-0.11418j | 0.21656+0.00000j | 0.02322+0.03526j | -0.01349+0.01293j | 0.02786-0.03082j | -0.01573+0.00945j | -0.10018+0.00079j | -0.00211-0.01527j
-0.07084-0.02386j | 0.02322-0.03526j | 0.21656+0.00000j | 0.00335+0.01821j | -0.01061-0.04092j | 0.00415-0.10010j | 0.01008-0.01557j | 0.00049+0.00024j
-0.06933+0.12470j | -0.01349-0.01293j | 0.00335-0.01821j | 0.24228-0.00000j | 0.09993-0.00717j | 0.00642+0.00146j | -0.00472-0.00445j | 0.05010-0.06913j
-0.10282+0.15727j | 0.02786+0.03082j | -0.01061+0.04092j | 0.09993+0.00717j | 0.21656+0.00000j | -0.01754-0.00533j | 0.01264+0.01377j | -0.00966+0.01149j
0.01616-0.05548j | -0.01573-0.00945j | 0.00415+0.10010j | 0.00642-0.00146j | -0.01754+0.00533j | 0.24235+0.00000j | -0.00570-0.00336j | 0.00109-0.00256j
0.02306+0.09088j | -0.10018-0.00079j | 0.01008+0.01557j | -0

In [10]:
head = data["head"]
head = head[:, :, 0, 0, 0, 0]

for row in range(head.shape[0]):
    row_data = []
    for col in range(head.shape[1]):
        row_data.append(f"{head[row, col]:.5f}")
    print(" | ".join(row_data))

1.00000-0.00000j | -0.05016+0.19131j | -0.11326+0.03814j | -0.12329-0.21733j | -0.17991-0.27518j | 0.02843+0.09821j | 0.04308-0.15990j | -0.00000-0.00000j
-0.05016-0.19131j | 1.00000+0.00000j | 0.04481+0.06806j | -0.24097+0.23319j | 0.05318-0.05884j | -0.28182+0.17010j | -0.03101-0.00090j | -0.00024-0.00175j
-0.11326-0.03814j | 0.04481-0.06806j | 1.00000-0.00000j | 0.06072+0.32644j | -0.02049-0.07906j | 0.00142-0.03097j | 0.18195-0.27839j | 0.00017+0.00008j
-0.12329+0.21733j | -0.24097-0.23319j | 0.06072-0.32644j | 1.00000-0.00000j | 0.03087-0.00288j | -0.04309-0.00855j | 0.03204+0.02999j | 0.01672-0.02048j
-0.17991+0.27518j | 0.05318+0.05884j | -0.02049+0.07906j | 0.03087+0.00288j | 1.00000+0.00000j | -0.31441-0.09672j | 0.22525+0.24866j | -0.00121+0.00144j
0.02843-0.09821j | -0.28182-0.17010j | 0.00142+0.03097j | -0.04309+0.00855j | -0.31441+0.09672j | 1.00000+0.00000j | 0.03747+0.02293j | 0.00345+0.00052j
0.04308+0.15990j | -0.03101+0.00090j | 0.18195+0.27839j | 0.03204-0.02999j | 0

In [11]:
wings = data["wings"]
wings = wings[:, :, 0, 0, 0, 0]

for row in range(wings.shape[0]):
    row_data = []
    for col in range(wings.shape[1]):
        row_data.append(f"{wings[row, col]:.5f}")
    print(" | ".join(row_data))

0.00000+0.00000j | -0.02333+0.08898j | -0.05267+0.01774j | 0.00336+0.00592j | -0.08366-0.12798j | -0.00077-0.00268j | -0.00117+0.00436j | -0.00000-0.00000j
-0.02333-0.08898j | 0.00000+0.00000j | 0.00232+0.00352j | -0.11867+0.11366j | 0.00279-0.00309j | -0.13844+0.08277j | 0.04012+0.00036j | -0.00030-0.00214j
-0.05267-0.01774j | 0.00232-0.00352j | 0.00000+0.00000j | 0.02968+0.15999j | -0.00106-0.00408j | -0.00173+0.04008j | 0.08877-0.13667j | 0.00007+0.00003j
0.00336-0.00592j | -0.11867-0.11366j | 0.02968-0.15999j | 0.00000+0.00000j | -0.03998+0.00328j | -0.00207-0.00041j | 0.00154+0.00144j | -0.01169+0.01591j
-0.08366+0.12798j | 0.00279+0.00309j | -0.00106+0.00408j | -0.03998-0.00328j | 0.00000+0.00000j | -0.15433-0.04658j | 0.11120+0.12111j | -0.00135+0.00161j
-0.00077+0.00268j | -0.13844-0.08277j | -0.00173-0.04008j | -0.00207+0.00041j | -0.15433+0.04658j | 0.00000+0.00000j | 0.00180+0.00110j | -0.00001+0.00069j
-0.00117-0.00436j | 0.04012-0.00036j | 0.08877+0.13667j | 0.00154-0.0014

In [12]:
body = data["body"]
body = body[:, :, 0, 0, 0, 0]

for row in range(body.shape[0]):
    row_data = []
    for col in range(body.shape[1]):
        row_data.append(f"{body[row, col]:.5f}")
    print(" | ".join(row_data))

0.26817-0.00000j | -0.08153+0.31100j | -0.18194+0.06128j | -0.20591-0.37437j | -0.29583-0.45252j | 0.04851+0.16597j | 0.06688-0.27254j | 0.01449+0.00175j
-0.08153-0.31100j | 0.12970+0.00000j | 0.16279+0.24722j | -0.31218+0.29588j | 0.19464-0.21534j | -0.36222+0.21700j | -0.03911+0.00136j | -0.01049-0.07603j
-0.18194-0.06128j | 0.16279-0.24722j | 0.12970-0.00000j | 0.07557+0.41917j | -0.07438-0.28700j | 0.00149-0.03910j | 0.22928-0.35978j | 0.00258+0.00123j
-0.20591+0.37437j | -0.31218-0.29588j | 0.07557-0.41917j | 0.04805+0.00000j | 0.03906-0.00221j | 0.14116+0.03079j | -0.10509-0.09890j | 0.01780-0.02828j
-0.29583+0.45252j | 0.19464+0.21534j | -0.07438+0.28700j | 0.03906+0.00221j | 0.12970-0.00000j | -0.40410-0.12148j | 0.29385+0.31446j | -0.04819+0.05735j
0.04851-0.16597j | -0.36222-0.21700j | 0.00149+0.03910j | 0.14116-0.03079j | -0.40410+0.12148j | 0.04797+0.00000j | -0.12414-0.07407j | -0.00501-0.00335j
0.06688+0.27254j | -0.03911-0.00136j | 0.22928+0.35978j | -0.10509+0.09890j | 